In [ ]:
import deepxde as dde
import numpy as np
import tensorflow as tf

# Definiujemy równanie różniczkowe
def ode(x, u):
    du_dx = dde.grad.jacobian(u, x)
    return du_dx - tf.cos(omega * x)

# Warunek początkowy
def boundary(_, on_initial):
    return on_initial

# Warunek początkowy
def func_boundary(x):
    return np.zeros((len(x), 1))

# Analityczne rozwiązanie do porównania
def exact_solution(x):
    return np.sin(omega * x) / omega

# Parametry problemu
omega = 1
geom = dde.geometry.Interval(-2 * np.pi, 2 * np.pi)

# Ustawienia dla pierwszego przypadku
bc = dde.DirichletBC(geom, func_boundary, boundary)
data = dde.data.PDE(geom, ode, bc, num_domain=200, num_boundary=2)

# Definicja sieci neuronowej
layer_size = [1] + [16] * 2 + [1]  # 2 warstwy ukryte, 16 neuronów każda
activation = "tanh"
initializer = "Glorot uniform"
net = dde.nn.FNN(layer_size, activation, initializer)

# Model
model = dde.Model(data, net)

# Kompilacja modelu
model.compile("adam", lr=0.001)

# Trening modelu
losshistory, train_state = model.train(epochs=50000)

# Testowanie modelu
x = geom.random_points(1000)
u_pred = model.predict(x)
u_exact = exact_solution(x)

# Wykresy wyników
import matplotlib.pyplot as plt

plt.figure()
plt.plot(x, u_pred, label='Predykcja sieci')
plt.plot(x, u_exact, label='Dokładne rozwiązanie')
plt.xlabel('x')
plt.ylabel('u(x)')
plt.legend()
plt.show()

# Wykres funkcji błędu
plt.figure()
plt.plot(losshistory.loss_train, label='Funkcja kosztu - trening')
plt.plot(losshistory.loss_test, label='Funkcja kosztu - test')
plt.xlabel('Epoki')
plt.ylabel('Koszt')
plt.legend()
plt.show()
